In [0]:
import pandas as pd
import os
from lxml import etree

In [1]:

# def createDicomPath(ID):
#     dirPart = os.path.dirname(ID)
#     basePart = os.path.basename(ID)
#     dirPart = os.path.normpath(dirPart.replace("NIFTI_SLICES","DICOM_STRUCTURED"))
#     basePartArray = basePart.split('__')[:-1]
#     basePart = os.path.normpath(str.join('/',basePartArray))
#     fullPath = os.path.normpath(str.join('/',[dirPart,basePart]))
#     return fullPath


# def findXmlName(DicomPath):
#     filename = os.listdir(DicomPath)[0]
#     seriesName = filename.split('_br_raw')[0]
#     subjectIdent = seriesName.split('_MR_')[0]
#     seriesName = seriesName.split('_MR_')[1]
#     while seriesName[0] == '_':
#         seriesName = seriesName[1:]
#     while seriesName[-1] == '_':
#         seriesName = seriesName[:-1]
#     snum = filename.split('_S')[-1]
#     snum = snum.split('_')[0]
#     snum = f'S{snum}'
#     Inum = filename.split('_I')[-1]
#     Inum = Inum.split('.dcm')[0]
#     Inum = f'I{Inum}'
#     xmlName = f'{subjectIdent}_{seriesName}_{snum}_{Inum}.xml'
#     return {'name':xmlName, 'description': seriesName}

# def getKey(info,key):
#     return info[key]

# def getWeightingFromXml(XML_Name, metaDataPath):
#     tree = etree.parse(os.path.join(metaDataPath,XML_Name))
#     for element in tree.iter():
#         try:
#             if element.attrib['term'] == 'Weighting':
#                 return element.text
#         except:
#             continue
#     return None

# 
# def main():
#     Labels = pd.read_csv('ADNI/FirstTest/DATA/Labels.txt',sep='\t',names=['ID','label','extra'])
#     Labels['dicomPath'] = Labels['ID'].apply(createDicomPath)
#     Labels['info'] = Labels['dicomPath'].apply(findXmlName)
#     Labels['xmlName']=Labels['info'].apply(getKey,key='name')
#     Labels['description']=Labels['info'].apply(getKey,key='description')
#     Labels['weighting'] = Labels['xmlName'].apply(getWeightingFromXml,metaDataPath='/trinity/home/r098375/DDS/ADNI/FirstTest/ADNI_train/metadata')
#     Labels.to_csv('ADNI/FirstTest/DATA/LabelsWithExtraInfo.csv',index=False)

In [19]:
CreateLabelsFrame = pd.read_csv('LabelsWithExtraInfo.csv')
CreateLabelsFrame = CreateLabelsFrame.drop(['dicomPath','info','xmlName'],axis=1)
CreateLabelsFrame.head()

,ID,label,extra,description,weighting
0,/trinity/home/r098375/DDS/data/ADNI/ADNI2/trai...,0,0,Axial_PD_T2_FSE,T2
1,/trinity/home/r098375/DDS/data/ADNI/ADNI2/trai...,0,0,Axial_PD_T2_FSE,T2
2,/trinity/home/r098375/DDS/data/ADNI/ADNI2/trai...,0,0,Axial_PD_T2_FSE,T2
3,/trinity/home/r098375/DDS/data/ADNI/ADNI2/trai...,0,0,Axial_PD_T2_FSE,PD
4,/trinity/home/r098375/DDS/data/ADNI/ADNI2/trai...,0,0,Axial_PD_T2_FSE,T2


In [20]:
CreateLabelsFrame['origDesc'] = CreateLabelsFrame['description']

In [21]:
CreateLabelsFrame = CreateLabelsFrame.dropna()

In [22]:
uniqueDesc = CreateLabelsFrame[CreateLabelsFrame['weighting']=='PD']['description'].unique()
print(uniqueDesc)

['Axial_PD_T2_FSE']


In [23]:
def renameDesc(desc):
    if 'localizer' in desc.lower():
        return 'localizer'
    if 'phase' in desc.lower():
        return 'phase'
    if 'field' in desc.lower():
        return 'fieldmap'
    if 'rage' in desc.lower():
        return 'T1w'
    if 'cal' in desc.lower():
        return 'calibration'
    if 'scout' in desc.lower():
        return 'scout'
    if 'fspgr' in desc.lower():
        return 'fspgr'
    if 'flair' in desc.lower():
        return 'T2-FLAIR'
    if 'star' in desc.lower():
        return 'T2star'
    if 'swi' in desc.lower():
        return 'swi'
    if 'hippo' in desc.lower():
        return 'hippo'
    if 'perf' in desc.lower() or 'pwi' in desc.lower():
        return 'pwi'
    if 'asl' in desc.lower():
        return 'Asl'
    if 'dti' in desc.lower():
        return 'dti'
    if 'cerebral_blood_flow' in desc.lower():
        return 'cbf'
    if 't2' in desc.lower():
        return 'T2w'


In [24]:
CreateLabelsFrame['description'] = CreateLabelsFrame['origDesc'].apply(renameDesc)

In [25]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"FS":4,"PD":5,
             "SWI":6, "GRE":7, "T2*":8, "DWI":9, "ADC":10, "BOLD":11,
             "FlowSensitve":12,"PWI":13,"Other":14}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    otherCategories = ['fieldmap','scout','hippo','calibration','phase','localizer','asl','dti','fspgr','cbf', 'nan', 'pasl','NotMri','pet','CT','e2']
    if description in otherCategories:
        return 14
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 5
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='FS':
        return 4
    if description =='pd':
        return 5
    if description =='swi' or description=='minIP':
        return 6
    if description.lower() =='gre':
        return 7
    if description =='T2star':
        return 8
    if description =='dwi':
        return 9
    if description =='adc':
        return 10
    if description =='bold':
        return 11
    if description =='angio': #flowsensitive
        return 12
    if description =='pwi':
        return 13
    return -1

In [26]:
CreateLabelsFrame['label'] = CreateLabelsFrame.apply(assignLabel,axis=1)

In [27]:
CreateLabelsFrame[CreateLabelsFrame['label']==-1]

,ID,label,extra,description,weighting,origDesc


In [28]:
CreateLabelsFrame = CreateLabelsFrame[CreateLabelsFrame['label']!=-1]

In [31]:
CreateLabelsFrame[['ID','label','extra']].to_csv('ADNI2_trainLabels.txt',index=False, sep='\t')

In [30]:
CreateLabelsFrame['label'].value_counts()

label
2    4925
5    4925
Name: count, dtype: int64